## Vertex Array Objects and Vertex Buffer Objects with PySide2

Welcome to VAO and VBO tutorial. 
Our goal in this tutorial is to show how to use more than 1 Vertex Array Object and Vertex Buffer Object in PySide2 while using the QtGui.QOpenGL* api.

Our resulting application would look like the following.

In [ ]:
import subprocess

subprocess.run(["python", "app.py"])

You should see two triangles pointing towards opposite directions, one should be blue and the other one should be red.

I assume that you have already followed through the first tutorial on drawing a triangle on opengl. So I won't be introducing all of helper functions that I have used there. 
Neither the application window which is basically the same window from that tutorial.

Now let's see the constructor of our OpenGL widget.

In [ ]:
class TriangleGL(QOpenGLWidget):
    def __init__(self, parent=None):
        QOpenGLWidget.__init__(self, parent)

        # shaders etc
        triangleTutoDir = os.path.dirname(__file__)
        shaderDir = os.path.join(triangleTutoDir, "shaders")
        availableShaders = ["triangle", "triangle2"]  # notice the use of 2 shaders
        self.shaders = {
            name: {
                "fragment": os.path.join(shaderDir, name + ".frag"),
                "vertex": os.path.join(shaderDir, name + ".vert")
            } for name in availableShaders
        }
        self.core = "--coreprofile" in QCoreApplication.arguments()

        # opengl data related
        self.context = QOpenGLContext()
        
        # each vertex array object has its own vertex buffer object
        self.vao1 = QOpenGLVertexArrayObject()
        self.vbo1 = QOpenGLBuffer(QOpenGLBuffer.VertexBuffer)
        
        self.vao2 = QOpenGLVertexArrayObject()
        self.vbo2 = QOpenGLBuffer(QOpenGLBuffer.VertexBuffer)

        # and each VAO-VBO couple has its own shader program 
        # to which we can attach different shaders 
        self.program1 = QOpenGLShaderProgram()
        
        self.program2 = QOpenGLShaderProgram()

        # some vertex data for corners of triangle
        # first triangle
        self.vertexData1 = np.array(
            [0.9, 0.9, 0.0,  # x, y, z
             0.9, 0.7, 0.0,  # x, y, z
             0.7, 0.9, 0.0],  # x, y, z
            dtype=ctypes.c_float
        )
        # second triangle
        self.vertexData2 = np.array(
            [-0.9, -0.9, 0.0,  # x, y, z
             -0.9, -0.7, 0.0,  # x, y, z
             -0.7, -0.9, 0.0],  # x, y, z
            dtype=ctypes.c_float
        )
        # triangle color
        self.triangleColor1 = QVector4D(1.0, 0.0, 0.0, 0.0)  # yellow triangle
        self.triangleColor2 = QVector4D(
            0.0, 0.0, 0.5, 0.0)  # not yellow triangle

As you can see we simply duplicated some of the objects. Remember:

- Each VAO uses its own VBO
- Each VAO-VBO couple uses its own shader program

How does this affect our initialization function `initializeGL` ? Let's see.

In [ ]:
        print('gl initial')
        print(self.getGlInfo())
        # create context and make it current
        self.context.create()
        self.context.aboutToBeDestroyed.connect(self.cleanUpGl)

        # initialize functions
        funcs = self.context.functions()
        funcs.initializeOpenGLFunctions()
        funcs.glClearColor(1, 1, 1, 1)

This should all be familiar to you by now so we are skipping the explanation.

In [ ]:
 # deal with shaders
        # first shader
        shaderName = "triangle"
        vshader = self.loadVertexShader(shaderName)
        fshader = self.loadFragmentShader(shaderName)

        # creating shader program
        self.program1 = QOpenGLShaderProgram(self.context)
        self.program1.addShader(vshader)  # adding vertex shader
        self.program1.addShader(fshader)  # adding fragment shader

        # bind attribute to a location
        self.program1.bindAttributeLocation("aPos", 0)

        # link shader program1
        isLinked = self.program1.link()
        print("shader program1 is linked: ", isLinked)

        # bind the program1
        self.program1.bind()

        # specify uniform value
        colorLoc = self.program1.uniformLocation("color")
        self.program1.setUniformValue(colorLoc,
                                     self.triangleColor1)


This by itself should also be familiar.

In [ ]:

        # second shader
        shaderName = "triangle2"
        vshader = self.loadVertexShader(shaderName)
        fshader = self.loadFragmentShader(shaderName)

        #
        self.program2 = QOpenGLShaderProgram(self.context)
        self.program2.addShader(vshader)  # adding vertex shader
        self.program2.addShader(fshader)  # adding fragment shader

        # bind attribute to a location
        self.program2.bindAttributeLocation("aPos", 0)

        # link shader program2
        isLinked = self.program2.link()
        print("shader program2 is linked: ", isLinked)

        # bind the program2
        self.program2.bind()

        # specify uniform value
        colorLoc = self.program2.uniformLocation("color")
        self.program2.setUniformValue(colorLoc,
                                     self.triangleColor2)


Here is a difference. Before we pass on to dealing with specific vao-vbo related to shader program, we arrange the second shader program. Then deal with the vao-vbo, as we see below.

In [ ]:

        # vao
        isVao = self.vao1.create()
        vaoBinder = QOpenGLVertexArrayObject.Binder(self.vao1)

        # vbo
        isVbo = self.vbo1.create()
        isBound = self.vbo1.bind()

        # check if vao and vbo are created
        print('vao created: ', isVao)
        print('vbo created: ', isVbo)

        floatSize = ctypes.sizeof(ctypes.c_float)

        # allocate space on buffer
        self.vbo1.allocate(self.vertexData1.tobytes(),
                           floatSize * self.vertexData1.size)
        funcs.glEnableVertexAttribArray(0)
        nullptr = VoidPtr(0)
        funcs.glVertexAttribPointer(0,
                                    3,
                                    int(pygl.GL_FLOAT),
                                    int(pygl.GL_FALSE),
                                    3 * floatSize,
                                    nullptr)
        self.vbo1.release()
        vaoBinder = None


Notice that at the and we unbind the vao-vbo couple that concerned the first triangle data. 

Now we pass on to the second one.

In [ ]:
# second triangle vao vbo
        # vao
        isVao = self.vao2.create()
        vaoBinder = QOpenGLVertexArrayObject.Binder(self.vao2)

        # vbo
        isVbo = self.vbo2.create()
        isBound = self.vbo2.bind()

        # check if vao and vbo are created
        print('vao created: ', isVao)
        print('vbo created: ', isVbo)

        floatSize = ctypes.sizeof(ctypes.c_float)

        # allocate space on buffer
        self.vbo2.allocate(self.vertexData2.tobytes(),
                           floatSize * self.vertexData2.size)
        funcs.glEnableVertexAttribArray(0)
        nullptr = VoidPtr(0)
        funcs.glVertexAttribPointer(0,
                                    3,
                                    int(pygl.GL_FLOAT),
                                    int(pygl.GL_FALSE),
                                    3 * floatSize,
                                    nullptr)
        self.vbo2.release()
        self.program2.release()

As usual at the end we release the data we were dealing with.

This concludes the initialization part. Let's see all of the function.

In [ ]:

    def initializeGL(self):
        print('gl initial')
        print(self.getGlInfo())
        # create context and make it current
        self.context.create()
        self.context.aboutToBeDestroyed.connect(self.cleanUpGl)

        # initialize functions
        funcs = self.context.functions()
        funcs.initializeOpenGLFunctions()
        funcs.glClearColor(1, 1, 1, 1)

        # deal with shaders
        # first shader
        shaderName = "triangle"
        vshader = self.loadVertexShader(shaderName)
        fshader = self.loadFragmentShader(shaderName)

        # creating shader program
        self.program1 = QOpenGLShaderProgram(self.context)
        self.program1.addShader(vshader)  # adding vertex shader
        self.program1.addShader(fshader)  # adding fragment shader

        # bind attribute to a location
        self.program1.bindAttributeLocation("aPos", 0)

        # link shader program1
        isLinked = self.program1.link()
        print("shader program1 is linked: ", isLinked)

        # bind the program1
        self.program1.bind()

        # specify uniform value
        colorLoc = self.program1.uniformLocation("color")
        self.program1.setUniformValue(colorLoc,
                                     self.triangleColor1)

        # second shader
        shaderName = "triangle2"
        vshader = self.loadVertexShader(shaderName)
        fshader = self.loadFragmentShader(shaderName)

        #
        self.program2 = QOpenGLShaderProgram(self.context)
        self.program2.addShader(vshader)  # adding vertex shader
        self.program2.addShader(fshader)  # adding fragment shader

        # bind attribute to a location
        self.program2.bindAttributeLocation("aPos", 0)

        # link shader program2
        isLinked = self.program2.link()
        print("shader program2 is linked: ", isLinked)

        # bind the program2
        self.program2.bind()

        # specify uniform value
        colorLoc = self.program2.uniformLocation("color")
        self.program2.setUniformValue(colorLoc,
                                     self.triangleColor2)

        # self.useShader("triangle")

        # deal with vao and vbo

        # create vao and vbo

        # vao
        isVao = self.vao1.create()
        vaoBinder = QOpenGLVertexArrayObject.Binder(self.vao1)

        # vbo
        isVbo = self.vbo1.create()
        isBound = self.vbo1.bind()

        # check if vao and vbo are created
        print('vao created: ', isVao)
        print('vbo created: ', isVbo)

        floatSize = ctypes.sizeof(ctypes.c_float)

        # allocate space on buffer
        self.vbo1.allocate(self.vertexData1.tobytes(),
                           floatSize * self.vertexData1.size)
        funcs.glEnableVertexAttribArray(0)
        nullptr = VoidPtr(0)
        funcs.glVertexAttribPointer(0,
                                    3,
                                    int(pygl.GL_FLOAT),
                                    int(pygl.GL_FALSE),
                                    3 * floatSize,
                                    nullptr)
        self.vbo1.release()
        vaoBinder = None

        # second triangle vao vbo
        # vao
        isVao = self.vao2.create()
        vaoBinder = QOpenGLVertexArrayObject.Binder(self.vao2)

        # vbo
        isVbo = self.vbo2.create()
        isBound = self.vbo2.bind()

        # check if vao and vbo are created
        print('vao created: ', isVao)
        print('vbo created: ', isVbo)

        floatSize = ctypes.sizeof(ctypes.c_float)

        # allocate space on buffer
        self.vbo2.allocate(self.vertexData2.tobytes(),
                           floatSize * self.vertexData2.size)
        funcs.glEnableVertexAttribArray(0)
        nullptr = VoidPtr(0)
        funcs.glVertexAttribPointer(0,
                                    3,
                                    int(pygl.GL_FLOAT),
                                    int(pygl.GL_FALSE),
                                    3 * floatSize,
                                    nullptr)
        self.vbo2.release()
        self.program2.release()

Now let's see how the actual drawing occurs.

In [ ]:
    def paintGL(self):
        "drawing loop"
        funcs = self.context.functions()

        # clean up what was drawn
        funcs.glClear(pygl.GL_COLOR_BUFFER_BIT)

        # actual drawing
        
        # bind the object you want to draw
        vaoBinder = QOpenGLVertexArrayObject.Binder(self.vao1)
        # activate its shader
        self.program1.bind()
        # presto
        funcs.glDrawArrays(pygl.GL_TRIANGLES,  # mode
                           0,  # first
                           3)  # count
        # unbind the object you've just drawn
        vaoBinder = None
        # and release its shader program
        self.program1.release()
        
        # now bind the next object you would like to draw
        vaoBinder = QOpenGLVertexArrayObject.Binder(self.vao2)
        
        # activate its shaders
        self.program2.bind()
        # presto!
        funcs.glDrawArrays(pygl.GL_TRIANGLES,  # mode
                           0,  # first
                           3)  # count
        # unbind the object you've just drawn
        vaoBinder = None
        # release the program
        self.program2.release()


And that's it. Now you know what to do if you need multiple VAOs-VBOs in your code. In most cases, you would require such a thing when you need different shaped objects, like a rectangle and a circle at the same time for example.